In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
events_data = pd.read_csv('datasets/df_concat6.csv')
events_data.head()

,Month,Area Affected,NERC Region,Event Type,Demand Loss (MW),Number of Customers Affected,Year,Date/Time Event Began,Date/Time Restoration,Event Duration (hours)
0,January,Washington,WECC,System Operations,0.0,NaN,2019,2019-01-05 13:19:00,2019-01-05 15:07:00,1.80
1,January,Washington,WECC,Severe Weather,230.0,230000.0,2019,2019-01-06 03:00:00,2019-01-09 07:00:00,4.00
2,January,California,WECC,Severe Weather,300.0,90382.0,2019,2019-01-06 17:56:00,2019-01-06 21:52:00,3.93
3,January,Washington,WECC,Severe Weather,NaN,230000.0,2019,2019-01-06 01:00:00,2019-01-06 12:00:00,11.00
4,January,Michigan,RF,System Operations,0.0,0.0,2019,2019-01-07 20:57:00,2019-01-07 21:32:00,0.58
